<a href="https://colab.research.google.com/github/boywaiter/nlp_notes/blob/master/nlp_notes02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2 Linear Text Claasification

**text classification**: given a text document, assign it a discrete label $y\in\mathcal{Y}$, where $\mathcal Y$ is the set of possible labels.

## 2.1 The bag of words
A document or an instance is represented as a column vector, $\boldsymbol x=[0, 1, 1, 0, 0, 2, 0, 1, 13, 0 ,\ldots]^\textrm{T}$, where $x_j$ is the count of the $j$-th word. The length of $\boldsymbol x$ is $V=|\mathcal V|$, the size of vocabulary.

**Bag of words**: contains only word count information, no order information of words.

**Weights** $\boldsymbol \theta$ assigns for each word a score, measuring the compatability with the label.

To predict the label $\hat y$ for a given $\boldsymbol x$, we compute a score $\Psi(\boldsymbol x, y)$ for each $y\in \mathcal Y$,
$$
\Psi (\boldsymbol x, y)= \mathbf \theta \cdot \boldsymbol f(\boldsymbol x, y)=\sum_{j=1}^V \theta_j\cdot f_j(\boldsymbol x,y) \tag{2.1}
$$
where, it may seem awkward, the $\boldsymbol f(\boldsymbol x, y)$ returns a column vector of length $K\times V$ and with $(K-1)\times V$ zeros,  
$$
{\boldsymbol{f}(\boldsymbol{x}, y=1)=[\boldsymbol{x} ; \underbrace{0 ; 0 ; \ldots ; 0}_{(K-1)\times V}] }\\ \tag{2.3}
$$
$$
{\boldsymbol{f}(\boldsymbol{x}, y=2)=[\underbrace{0 ; 0 ; \ldots ; 0 }_{V};\boldsymbol{x} ; \underbrace{0 ; 0 ; \ldots ; 0}_{(K-2)\times V}]}\tag{2.4}
$$
$$
{\boldsymbol{f}(\boldsymbol{x}, y=K)=[\underbrace{0 ; 0 ; \ldots ; 0}_{(K-1) \times V} ; \boldsymbol{x}]}\tag{2.5}
$$
The weights $\boldsymbol \theta$ is thus a column vector of the length $K\times V$, i.e., $\boldsymbol \theta\in \mathbb R^{KV}$, where $\theta_{(k-1)*V+1:kV}$ are for label $y=k$. 

We predict the label $\hat y$ with
$$
\hat y =\mathop{\text{argmax}}\limits_{y\in \mathcal Y} \Psi(\boldsymbol x,y)\tag{2.6}
$$
$$
\Psi(\boldsymbol x,y )=\boldsymbol \theta\cdot\boldsymbol f(\boldsymbol x,y)\tag{2.7}
$$
It is common to add an **offset feature** at the end of $\boldsymbol x$, which is always 1. The weight for it can be thought of as a bias.




In [0]:
import torch

In [0]:
K=3
V=5
x=torch.randint(10,(V,1))
x=torch.cat((x,torch.ones(1,1,dtype=torch.long)),0)#offset features
weights=torch.ones(K*(V+1))

In [0]:
def feature_function(x,y):
  return {(y-1)*(V+1)+i:x[i] for i in range(0,V+1)}

In [0]:
print(feature_function(x,2))

{6: tensor([7]), 7: tensor([5]), 8: tensor([8]), 9: tensor([7]), 10: tensor([1]), 11: tensor([1])}


In [0]:
def compute_score(x,y,weights):
  total=0
  for feature, count in feature_function(x,y).items():
    total+=weights[feature] * count
  return total

In [0]:
print(x)
print(weights)
total=compute_score(x,2,weights)
print(total)

tensor([[7],
        [5],
        [8],
        [7],
        [1],
        [1]])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([29.])


## 2.2 Naive Bayes
$\textrm{p}(\boldsymbol x, y)$ is the **joint probability** of a text instance. A dataset of $N$ labeled instances is $\{(\boldsymbol x^{(i)}, y^{(i)})\}_{i=1}^N$. We assume these instances are **independent and identically distributed (IID)**, i.e., $\textrm{p}(\boldsymbol x, y)=\prod_{i=1}^N \textrm{p}(\boldsymbol x^{(i)}, y^{(i)})$.

**Maximum Likelihood Estimation**:
$$
\begin{eqnarray}
\hat y &=& \mathop{\text{argmax}}\limits_\boldsymbol \theta \textrm{p}(\boldsymbol x, y;\boldsymbol \theta)\tag{2.8}\\
&=&\mathop{\text{argmax}}\limits_\boldsymbol \theta\prod_{i=1}^N \textrm{p}(\boldsymbol x^{(i)}, y^{(i)};\boldsymbol \theta) \tag{2.9}\\
&=&\mathop{\text{argmax}}\limits_\boldsymbol \theta\sum_{i=1}^N \log\textrm{p}(\boldsymbol x^{(i)}, y^{(i)};\boldsymbol \theta)\tag{2.10}
\end{eqnarray}
$$
The probability $\textrm{p}(\boldsymbol x^{(i)}, y^{(i)};\boldsymbol \theta)$ is defined through a generative model — an idealized
random process that has generated the observed data. The joint probability is factored using the chain rule,
$$
\textrm{p}_{X,Y}(\boldsymbol x^{(i)}, y^{(i)};\boldsymbol \theta)=\textrm{p}_{X|Y}(\boldsymbol x^{(i)}| y^{(i)};\boldsymbol \phi)\times \textrm{p}_{Y}(y^{(i)};\boldsymbol \mu)\tag{2.11}
$$
where $\boldsymbol \theta=\{\boldsymbol \mu, \boldsymbol \phi\}$.

- The distribution for $\text p_{X|Y}$ is the **multinomial**,
$$
\begin{eqnarray}
\text p_\text{mult}(\boldsymbol x; \boldsymbol \phi)&=&B(\boldsymbol x)\prod_{j=1}^V \phi_j^{x_j}\tag{2.12}\\
B(\boldsymbol x)&=&\frac{(\sum_{j=1}^V x_j)!}{\prod_{j=1}^V(x_j)!}\tag{2.13}
\end{eqnarray}
$$
where $\phi_j$ is the probability $x_j$ occurs in $\boldsymbol x$.
- The distribution for $\text p_{Y}$ is the **categorical**, $\text p(y;\boldsymbol \mu)=\mu_y$ with $\boldsymbol \mu=[\mu_1; \mu_2; \ldots, \mu_K]$ and $\sum_{j=1}^K \mu_j=1$.

### 2.2.1 Types and tokens

If we use a sequence of tokens $\boldsymbol w=(w_1, w_2, \ldots, w_M)$ instead of types $\boldsymbol x=\{x_j|x_j\in\{1, 2, \ldots, M\}\}$ to represent a text instance, then the probability for it is a product of categorical probabilities, e.g., $\text p(w_m)=\phi_m$. 
$$
\text p(\boldsymbol w;\boldsymbol \phi)=\prod_{m=1}^M \text p(w_m;\phi_m)=\prod_{m=1}^M \phi_m=\prod_{j=1}^V \phi_j^{x_j}
$$
### 2.2.2 Prediction
$$
\begin{eqnarray}
\hat y &=& \mathop{\text{argmax}}\limits_y \log \textrm{p}(\boldsymbol x, y;\boldsymbol \mu, \boldsymbol \phi)\tag{2.14}\\
&=& \mathop{\text{argmax}}\limits_y \log \textrm{p}(\boldsymbol x| y;\boldsymbol \phi)+\log\textrm{p}(y;\boldsymbol \mu)\tag{2.15}\\
\end{eqnarray}
$$
$$
\begin{eqnarray}
\log \textrm{p}(\boldsymbol x| y;\boldsymbol \phi)+\log\textrm{p}(y;\boldsymbol \mu)&=&\log \left[B(\boldsymbol x)\prod_{j=1}^V \phi_j^{x_j}\right]+\log \mu_y\tag{2.16}\\
&=&\log B(\boldsymbol x)+\sum_{j=1}^V x_j\log \phi_j+\log \mu_y\tag{2.17}\\
&=&\log B(\boldsymbol x)+\boldsymbol \theta \cdot \boldsymbol f(\boldsymbol  x, y)\tag{2.18}
\end{eqnarray}
$$
where 
$$\boldsymbol \theta=[\boldsymbol \theta^{(1)}; \boldsymbol \theta^{(2)}; \ldots; \boldsymbol \theta^{(K)}]\tag{2.19}$$
$$\boldsymbol \theta^{(y)}=[\log \phi_{y,1}; \log \phi_{y,2}; \ldots ; \log \phi_{y,V};\log \mu_y]\tag{2.20}$$
and $\boldsymbol f(\boldsymbol  x, y)$ is shown in Equations 2.3-2.5. This is a key point: through this notation, we have converted the problem of computing the log-likelihood for a document-label pair (x; y) into the computation of a vector inner product.
### 2.2.3 Estimation
The parameters of the categorical and multinomial distributions have a simple interpretation: they are vectors of expected frequencies for each possible event.
$$
\phi_y,j=\frac{count(y,j)}{\sum_{j'=1}^V count(y,j')}=\frac{\sum_{i:y^{(i)}=y}x^{(i)}_j}{\sum_{j'=1}^V \sum_{i:y^{(i)}=y}x^{(i)}_{j'}}
$$
where $count(y,j)$ refers to the count of word $j$ in documents labeled as $y$.

Equation 2.21 defines the relative frequency estimate for $\phi$. It can be justified as a **maximum likelihood estimate**.
$$
\mathcal{L}(\boldsymbol \phi, \boldsymbol \mu)=\sum_{i=1}^N \log \text p_{\text{mult}}(\boldsymbol x^{(i)}|y^{(i)};\phi_{y^{(i)}})+\log \text p_{\text{cat}}(y^{(i)};\mu_{y^{(i)}})\tag{2.22}
$$
$$
\mathcal{L}(\boldsymbol \phi)=\sum_{i=1}^N \log \text p_{\text{mult}}(\boldsymbol x^{(i)}|y^{(i)};\phi_{y^{(i)}})=\sum_{i=1}^N \left(\log B(\boldsymbol x^{(i)})+\sum_{j=1}^Vx^{(i)}_j\log\phi_{y^{(i)}, j}\right)\tag{2.23}
$$
$$
\sum_{j=1}^V \phi_{y,j}=1\text{ } \forall y \tag{2.24}
$$
$$
\ell(\phi_y)=\sum_{i:y^{(i)}=y}^N\sum_{j=1}^V x^{(i)}_j\log\phi_{y, j}-\lambda(\sum_{j=1}^V \phi_{y,j}-1).\tag{2.25}
$$
$$
\frac{\partial \ell(\phi_y)}{\partial \phi_{y,j}}=\sum_{i:y^{(i)}=y}^N x^{(i)}_j/\phi_{y, j}-\lambda\tag{2.26}
$$
$$
\lambda\phi_{y, j}=\sum_{i:y^{(i)}=y}^N x^{(i)}_j\tag{2.27}
$$
$$
\phi_{y, j}\propto\sum_{i:y^{(i)}=y}^N x^{(i)}_j=\sum_{i=1}^N \delta (y^{(i)}=y)x^{(i)}_j=count(y, j)\tag{2.28}
$$
$$
\mathcal{L}(\boldsymbol \mu)=\sum_{i=1}^N \log \text p_{\text{cat}}(y^{(i)};\mu_{y^{(i)}})=\sum_{i=1}^N \log \mu_{y^{(i)}}=\sum_{y\in\mathcal{Y}}\sum_{i=1}^N \delta (y^{(i)}=y)\log \mu_y $$
$$
\sum_{y\in\mathcal{Y}}\mu_y=1
$$
$$
\ell(\boldsymbol\mu)=\sum_{y\in\mathcal{Y}}\sum_{i=1}^N \delta (y^{(i)}=y)\log \mu_y -\lambda(\sum_{y\in\mathcal{Y}}\mu_y-1)
$$
$$
\frac{\partial \ell(\boldsymbol\mu)}{\partial \mu_y}=\sum_{i=1}^N \delta (y^{(i)}=y)/\mu_y -\lambda
$$
$$
\mu_y\propto \sum_{i=1}^N \delta (y^{(i)}=y)
$$
### 2.2.4 Smoothing
Given $\phi_{FICTION, molybdenum}=0$, $\log \phi_{FICTION, molybdenum}=-\infty$, which in turn makes $\text p(FICTION|\boldsymbol x)=0$.
 

## 2.3 Discriminative learning
**Descriminative learning**, comparing to generative learning, avoid learning the joint probability of the training instance and its label, focuses directly on predicting the label.

### 2.3.1 Perceptron
**Online learning**: the classifier weights update after every example.

**Batch learning**: computes statistics over the entire dataset and then sets the weights in a single operation.




In [0]:
Word_count_max=10
N=10
V=5
K=2
X=torch.randint(low=1,high=Word_count_max,size=(N,V))
X=torch.cat((X,torch.ones(N,1,dtype=torch.long)),1)
y=torch.randint(0,K,size=(N,1))

In [10]:
print(X)

tensor([[2, 1, 3, 2, 9, 1],
        [5, 7, 3, 9, 7, 1],
        [2, 9, 3, 7, 4, 1],
        [6, 5, 3, 8, 1, 1],
        [2, 1, 9, 9, 3, 1],
        [8, 1, 5, 3, 1, 1],
        [5, 9, 2, 3, 5, 1],
        [3, 8, 8, 2, 2, 1],
        [1, 8, 3, 1, 7, 1],
        [7, 7, 8, 5, 8, 1]])


In [0]:
import numpy as np
def feature_function_binary(x,y):
  dict1={y*(V+1)+i:x[i].item() for i in range(0,V+1)}  
  dict2={(1-y)*(V+1)+i:0 for i in range(0,V+1)}
  dict3={}
  dict3.update(dict1)
  dict3.update(dict2)
  return dict3
def update_theta(theta,D1,D2):
  for feature, count in D1.items():
    #print("feature=",feature," count=",count)
    theta[feature]+=count-D2[feature]
  return theta
def compute_score_binary(x,y,weights):
  total=0
  for feature, count in feature_function_binary(x,y).items():
    #print("feature=",feature)
    #print("count=",count)
    total+=weights[feature] * count
  return total
def perceptron(X,y):
  t=0
  theta=torch.zeros(K*(V+1),dtype=torch.float)
  #print("theta=",theta)
  times=100
  while(t<times):
    i= np.random.randint(0,N,1).item()
    x=X[i]    
    #print("x=",x)
    hat_y=0 if compute_score_binary(x,0,theta).item()>=compute_score_binary(x,1,theta).item() else 1
    #print("hat_y=",hat_y,"y[i]=",y[i].item())
    if(hat_y!=y[i].item()):
      theta=update_theta(theta,feature_function_binary(x,y[i].item()),
                         feature_function_binary(x,hat_y))
    t+=1
  return theta

In [12]:
theta = perceptron(X,y)
print(theta)

feature= 6  count= 2
feature= 7  count= 1
feature= 8  count= 3
feature= 9  count= 2
feature= 10  count= 9
feature= 11  count= 1
feature= 0  count= 0
feature= 1  count= 0
feature= 2  count= 0
feature= 3  count= 0
feature= 4  count= 0
feature= 5  count= 0
feature= 0  count= 5
feature= 1  count= 9
feature= 2  count= 2
feature= 3  count= 3
feature= 4  count= 5
feature= 5  count= 1
feature= 6  count= 0
feature= 7  count= 0
feature= 8  count= 0
feature= 9  count= 0
feature= 10  count= 0
feature= 11  count= 0
feature= 6  count= 8
feature= 7  count= 1
feature= 8  count= 5
feature= 9  count= 3
feature= 10  count= 1
feature= 11  count= 1
feature= 0  count= 0
feature= 1  count= 0
feature= 2  count= 0
feature= 3  count= 0
feature= 4  count= 0
feature= 5  count= 0
feature= 0  count= 7
feature= 1  count= 7
feature= 2  count= 8
feature= 3  count= 5
feature= 4  count= 8
feature= 5  count= 1
feature= 6  count= 0
feature= 7  count= 0
feature= 8  count= 0
feature= 9  count= 0
feature= 10  count= 0
featur